## Whiskey Ratings Query and Data Parsing with Google API
- Description: Uses the Google Developer API to connect to Google Forms results for whiskey tasting inputs, cleans the data up in to a functional dataframe, and then combines previously isolated geolocation data from distilleries to parse together a dataframe for use in Data Visualization processes.
- Goal : Create a repository/database of information for use of merging whiskey tasting notes to for dashboard development. The main objective is to use this mined data to isolate lat/long data to plot on a map where the whiskey tasting notes tied to a distillery are located per request of tasters who want to see where they've tasted whiskey from.
- Author: Kimberly Gaddie
- Date Last Updated: 19 September 2022

### Step 0 - Import Required Packages

In [11]:
# Packages required to interact with Google 

from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

from df2gspread import df2gspread as d2g
import gspread

# Other Packages Required
import pandas as pd
from datetime import datetime

### Step 1 - Connect to Google Sheets Document

In [34]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'data/credentials.json' #Contains service account permissions and api_keys to access dataset

# Setting Parameters for pulling the ratings data
SAMPLE_SPREADSHEET_ID = 'INSERT_SPREADSHEET_NAME' #found via the url for the google sheet 
SAMPLE_RANGE_NAME = 'INSERT_SHEET_NAME OR RANGE' # the sheet name

#When using a service account to access the Google Sheet via their API use this functionality
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Building the required service and querying the desired worksheet to gather data contained on Google Sheets
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', []) 

The final product here is a list of all values contained within the Google Sheet which isn't really user friendly. Thus, we need to format it into a functional format. This is done in Step 2.

### Step 2 - Format received data into Pandas dataframe

In [14]:
header = result.get('values', [])[0]   # Assumes first line is header!
values = result.get('values', [])[1:]  # Everything else is data.

all_data = []
for col_id, col_name in enumerate(header):
    column_data = []
    for row in values:
        column_data.append(row[col_id])
    ds = pd.Series(data=column_data, name=col_name)
    all_data.append(ds)
df = pd.concat(all_data, axis=1)

df.head()

,Timestamp,Whiskey Connoisseur,Whiskey Type,Region,Distillery,Whiskey Name,"Age (Enter Years, if unknown type ""NAS"")",ABV (%),Cost ($ USD),On Hand,...,Herbal,Spicy,Floral,Fruity,Body,Heat/Proof,Linger,Char,Tannic,Notes
0,7/23/2022 12:33:17,Gaddie,Single Malt,Germany,Eifel Destillate,German Single Malt Whisky,8,46,75.99,No,...,0,4,1,4,5,0,5,0,3,This spirit was first aged for at least 4 year...
1,7/23/2022 12:41:51,Gaddie,Single Malt,Islay (Scotch),Bunnahabhain,Artemis,30,54.8,130,Yes,...,,,,,,,,,,Bunnahabhain 30 Year Old 1990 vintage Islay si...
2,7/23/2022 12:46:01,Gaddie,Single Malt,Islay (Scotch),Ardbeg,Supernova,NAS,53.8,180,Yes,...,,,,,,,,,,SN2019 probes the palate - revealing smoky mal...
3,7/23/2022 12:48:25,Gaddie,Single Malt,Islay (Scotch),Ardbeg,Galileo - 1999,12,49,450,Yes,...,,,,,,,,,,"One of 2012's most hotly-anticipated releases,..."
4,7/23/2022 12:58:57,Gaddie,Single Malt,Islands (Scotch),Tobermory,Ledaig - 1996 Vintage Sherry Cask FInish,19,46.3,154.99,Yes,...,1,4,3,2,4,4,5,1,2,Peat-dried malted barley is married with natur...


### Step 3 - Merge Distillery Location Data to Dataframe

In [15]:
# Importing Distillery Database Information

distill_db = pd.read_csv('distillery_data.csv')
distill_db = distill_db.drop(distill_db.columns[[0]],axis = 1) #dropping first column which had contained indexing from saving previously
distill_db.head()

,distillery,country,latitude,longitude
0,'t Koelschip,Netherlands,52.243498,5.634323
1,1000 Hills Distillery,Rwanda,-1.964663,30.064436
2,10th Mountain,United States,34.566618,-93.173342
3,10th Street Distillery,United States,37.345137,-121.892493
4,117° West Spirits,United States,41.500245,-81.699377


In [16]:
# merging geolocation data to tasting dataframe

tasting_df = df.merge(distill_db,how='left', left_on='Distillery', right_on='distillery')

In [17]:
tasting_df.head()

,Timestamp,Whiskey Connoisseur,Whiskey Type,Region,Distillery,Whiskey Name,"Age (Enter Years, if unknown type ""NAS"")",ABV (%),Cost ($ USD),On Hand,...,Body,Heat/Proof,Linger,Char,Tannic,Notes,distillery,country,latitude,longitude
0,7/23/2022 12:33:17,Gaddie,Single Malt,Germany,Eifel Destillate,German Single Malt Whisky,8,46,75.99,No,...,5,0,5,0,3,This spirit was first aged for at least 4 year...,Eifel Destillate,Germany,51.163818,10.447831
1,7/23/2022 12:41:51,Gaddie,Single Malt,Islay (Scotch),Bunnahabhain,Artemis,30,54.8,130,Yes,...,,,,,,Bunnahabhain 30 Year Old 1990 vintage Islay si...,Bunnahabhain,Scotland,56.786111,-4.114052
2,7/23/2022 12:46:01,Gaddie,Single Malt,Islay (Scotch),Ardbeg,Supernova,NAS,53.8,180,Yes,...,,,,,,SN2019 probes the palate - revealing smoky mal...,Ardbeg,Scotland,56.786111,-4.114052
3,7/23/2022 12:48:25,Gaddie,Single Malt,Islay (Scotch),Ardbeg,Galileo - 1999,12,49,450,Yes,...,,,,,,"One of 2012's most hotly-anticipated releases,...",Ardbeg,Scotland,56.786111,-4.114052
4,7/23/2022 12:58:57,Gaddie,Single Malt,Islands (Scotch),Tobermory,Ledaig - 1996 Vintage Sherry Cask FInish,19,46.3,154.99,Yes,...,4,4,5,1,2,Peat-dried malted barley is married with natur...,Tobermory,Scotland,56.786111,-4.114052


### Step 4 - Write off Final Dataframe Back to Google Sheets for Use

In [37]:
# Setting Parameters for pushing the expanded ratings data
spreadsheet_key = 'INSERT SPREADSHEET KEY HERE' #found via the url for the google sheet 
wks_name = 'INSERT SHEET NAME HERE' # the sheet name
gc = gspread.authorize(creds)
credentials = creds

In [ ]:
d2g.upload(tasting_df, spreadsheet_key, wks_name, credentials=creds, row_names=True)